In [ ]:
#从这里开始
#这里两个部分都是一样的 从同一张表格出发

import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

all_data=pd.read_csv('D:/new_date.csv',low_memory=False)
print(all_data.shape)

#这里把数据集打乱 防止人都出现在一起
all_data = all_data.sample(frac=1.0)
all_data = all_data.reset_index()
all_data_na = (all_data.isnull().sum() / len(all_data)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
print(missing_data.head)


#删除一个样本 也许会漏掉一个user_id 这里我的态度很坚决 漏就漏了 我赋值0.5
#因为假数据也是猜 为什么不猜一手大的？
all_data.drop(all_data[all_data['customer_province']=='柔佛'].index,inplace=True)
print(all_data.shape)


all_data['is_customer_rate'].fillna(0,inplace=True)
all_data['customer_gender'].fillna(0,inplace=True)
all_data['is_member_actived'].fillna(0,inplace=True)

#member_status 有点奇怪 只有1  所以填充的时候要附上一个2 
#all_data['member_status'].plot()
#子订单应付总金额不能填充0 请注意 实在是太多了所以删除掉

all_data['member_status'].fillna(2,inplace=True)
all_data.drop(columns=['order_detail_amount'],inplace=True)
all_data.drop(columns=['customer_city','customer_province'],inplace=True)
#把一些莫名其妙的drop
all_data.drop(columns=['level_0','Unnamed: 0'],inplace=True)

'''
#这个地方排序一下 后面就写的更舒服一些一定要注意 这里不排序 后面减就会有副职
#当然也可以在排序的时候再写
all_data.sort_values(by='order_pay_time',inplace=True)
'''

#下面查看缺失量
all_data_na = (all_data.isnull().sum() / len(all_data)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data.head

#日期处理
all_data['order_pay_time'] = pd.to_datetime(all_data['order_pay_time'])
all_data['order_pay_date'] = all_data['order_pay_time'].dt.date
##地区编码
from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()
all_data['new_location_code'] = label.fit_transform(all_data['new_location'])
print("日期处理完毕-地区编码也已经完成")
#这个时候先导入训练好的物品特征 
#通过制定聚类算法 已经打好了20种标签
#这里先不急着把他马上拼接上去
#而是要给每一个客人买过的id先打上标签让每一行都有商品的特征
goods_feature=pd.read_csv('goods_feature.csv')
goods_feature.drop(columns='Unnamed: 0',inplace=True)
print("商品特征读取成功")



all_data=pd.merge(all_data,goods_feature[['goods_id','Goods_clustering_kind']],on='goods_id')
print(all_data.shape)
print('所有的前期处理准备完毕')

In [ ]:
#下面要对商品种类开始处理
#请注意脑袋不要乱  这个地方不要乱merge
#因为我需要的是每一种类的值
#其实按道理来说 这个处理应该要在商品种处理好,而不是放在这个地方处理

#商品种类平均价格
Goods_kind__price_mean=goods_feature.groupby('Goods_clustering_kind')['goods_price_mean'].agg({'Goods_kind__price_mean':np.mean})
print(Goods_kind__price_mean,'\n')

#商品种类的会员折扣
Goods_kind_member_discount_mean=goods_feature.groupby('Goods_clustering_kind')['goods_memeber_discount'].agg({'Goods_member_discount_mean':np.mean})
print(Goods_kind_member_discount_mean,'\n')

#商品种类的在库状态
Goods_kind_status_mean=goods_feature.groupby('Goods_clustering_kind')['goods_status_mean'].agg({'Goods_kind_status_mean':np.mean})
print(Goods_kind_status_mean,'\n')

#商品种类的平均在架时间
Goods_kind_date_time_mean=goods_feature.groupby('Goods_clustering_kind')['deta_time_mean'].agg({'Goods_kind_date_time_mean':np.mean})
print(Goods_kind_date_time_mean,'\n')

#商品种类的最大优惠了多少
Goods_kind_discount=goods_feature.groupby('Goods_clustering_kind')['goods_discount'].agg({'Goods_kind_discount':np.mean})
print(Goods_kind_discount,'\n')

#商品种类的性别
Goods_kind_gender=goods_feature.groupby('Goods_clustering_kind')['goods_gender'].agg({'Goods_kind_gender':np.mean})

print(Goods_kind_gender,'\n')

In [ ]:
def merge1(df):
    all_user=df
    for i in range(20):
        all_user['商品种类'+str(i)+'平均价格']=all_user['已经购买的商品种类—'+str(i)].apply(
                                                                        lambda x:Goods_kind__price_mean.values.flatten()[i] if x==1 else 0)
        all_user['商品种类'+str(i)+'的会员折扣']=all_user['已经购买的商品种类—'+str(i)].apply(
                                                                        lambda x:Goods_kind_member_discount_mean.values.flatten()[i] if x==1 else 0)
        all_user['商品种类'+str(i)+'的在库状态']=all_user['已经购买的商品种类—'+str(i)].apply(
                                                                        lambda x:Goods_kind_status_mean.values.flatten()[i] if x==1 else 0)
        all_user['商品种类'+str(i)+'的平均在架时间']=all_user['已经购买的商品种类—'+str(i)].apply(
                                                                        lambda x:Goods_kind_date_time_mean.values.flatten()[i] if x==1 else 0)
        all_user['商品种类'+str(i)+'的最大优惠了多少']=all_user['已经购买的商品种类—'+str(i)].apply(
                                                                        lambda x:Goods_kind_discount.values.flatten()[i] if x==1 else 0)
        all_user['商品种类'+str(i)+'的性别']=all_user['已经购买的商品种类—'+str(i)].apply(
                                                                        lambda x:Goods_kind_gender.values.flatten()[i] if x==1 else 0)
    return all_user




def make_feature_and_label(date1,date2,isSubmit):
    
    # #下面开始聚合用户特征 这个函数一举两得
    #为什么？因为每一个用户只能出现一次
    #而且通过这个特征还得到了客户的购买次数，写的很漂亮！
    # 统计这个用户出现了多少次
    date1['count'] = 1
    customer_id = date1.groupby(['customer_id'],as_index=False)['count'].agg({'count':'count'})
    
   
    #第一步开始聚合商品
    date1['购买了商品+'] = list(map(lambda x: '已经购买的商品种类—' + str(x),date1.Goods_clustering_kind))
    temp = pd.crosstab(date1['customer_id'],date1['购买了商品+']).reset_index()
    #要注意 两条线
    #一条主线是output
    #另一条主线是不能动的data1 当然也有data2
    #然后
    output=pd.merge(customer_id,temp,on='customer_id')
    output=merge1(output)
    print('已经把购买商品的属性加上去了')
    
    
    
    #1
    q=date1['order_pay_time'].apply(lambda x: x.month)
    date1['购买了商品+'] = list(map(lambda x:str(x)+'月份购买',q))
    temp = pd.crosstab(date1['customer_id'],date1['购买了商品+']).reset_index()
    del date1['购买了商品+']
    output = pd.merge(output, temp, on=['customer_id'], how='left')  #
    print("把月份属性加上了")
    
    #如果是训练集的话是需要管的
    #如果是全集的话是不用管的
    if isSubmit ==False:
        output['8月份购买']=0
        output['9月份购买']=0
        output['10月份购买']=0
        output['11月份购买']=0
        output['12月份购买']=0
    
    q=date1['order_pay_time'].apply(lambda x: x.day)
    date1['购买了商品+'] = list(map(lambda x:str(x)+'天购买',q))
    temp = pd.crosstab(date1['customer_id'],date1['购买了商品+']).reset_index()
    del date1['购买了商品+']
    output = pd.merge(output, temp, on=['customer_id'], how='left')  #
    print('把天的属性加上了')

    # 2,用户各个每月购买量占比（总的）
    for i in range(1,8):
        output['Month_'+str(i)+'_rate'] = output[str(i)+'月份购买'] / output['count']
    # 3,用户各天购买率占比（总的）
    for i in range(1,32):
        output['day_'+str(i)+'_rate']= output[str(i)+'天购买'] / output['count']
    
    #result['day_30_rate'] = result['31天购买'] / result['count']

    print("此时输出已经加上每天的以及每个月的购买率")
    
    
    
    #用户总购买的商品数目五个属性
    customer_buy_goods_num = date1.groupby(['customer_id'], as_index=False)['order_total_num'].agg(
                                                                                  {  'order_total_num_max':'max',
                                                                                     'order_total_num_mean':'mean',
                                                                                     'order_total_num_min':'min',
                                                                                     'order_total_num_sum':'sum',
                                                                                     'order_total_num_median':'median'}
                                                                                                  )
    print('用户总购买的商品数目五个属性输出ok')
    #父订单商品总金额五个属性
    customer_total_pay = date1.groupby(['customer_id'], as_index=False)['order_amount'].agg(
                                                                                    {'order_amount_max':'max',
                                                                                     'order_amount_mean':'mean',
                                                                                     'order_amount_min':'min',
                                                                                     'order_amount_sum':'sum',
                                                                                     'order_amount_median':'median'})
    
    print('父订单商品总金额五个属性ok')
    #父订单实付金额
    customer_real_pay = date1.groupby(['customer_id'], as_index=False)['order_total_payment'].agg(
                                                                                    {'order_total_payment_max':'max',
                                                                                     'order_total_payment_mean':'mean',
                                                                                     'order_total_payment_min':'min',
                                                                                     'order_total_payment_sum':'sum',
                                                                                     'order_total_payment_median':'median'})
    print('父订单实付金额ok')
    #父订单优惠金额
    customer_happy_pay = date1.groupby(['customer_id'], as_index=False)['order_total_discount'].agg(
                                                                                    {'order_total_discount_max':'max',
                                                                                     'order_total_discount_mean':'mean',
                                                                                     'order_total_discount_min':'min',
                                                                                     'order_total_discount_sum':'sum',
                                                                                     'order_total_discount_median':'median'})
    print('父订单优惠金额ok')
    #父订单包含的子订单数量的五个属性
    customer_number = date1.groupby(['customer_id'], as_index=False)['order_count'].agg(
                                                                                    {'order_count_max':'max',
                                                                                     'order_count_mean':'mean',
                                                                                     'order_count_min':'min',
                                                                                     'order_count_sum':'sum',
                                                                                     'order_count_median':'median'})
    print('父订单包含的子订单数量的五个属性ok')
    #子订单实付金额
    customer_son_real_pay = date1.groupby(['customer_id'], as_index=False)['order_detail_payment'].agg(
                                                                                    {'order_detail_payment_max':'max',
                                                                                     'order_detail_payment_mean':'mean',
                                                                                     'order_detail_payment_min':'min',
                                                                                     'order_detail_payment_sum':'sum',
                                                                                     'order_detail_payment_median':'median'})
    print('子订单实付金额ok')
    #子订单优惠金额
    customer_son_happy_pay = date1.groupby(['customer_id'], as_index=False)['order_detail_discount'].agg(
                                                                                    {'order_detail_discount_max':'max',
                                                                                     'order_detail_discount_mean':'mean',
                                                                                     'order_detail_discount_min':'min',
                                                                                     'order_detail_discount_sum':'sum',
                                                                                     'order_detail_discount_median':'median'})
    print('子订单优惠金额ok')
    
    
    output=pd.merge(output,customer_buy_goods_num,on='customer_id')
    output=pd.merge(output,customer_total_pay,on='customer_id')
    output=pd.merge(output,customer_real_pay,on='customer_id')
    output=pd.merge(output,customer_happy_pay,on='customer_id')
    output=pd.merge(output,customer_number,on='customer_id')
    output=pd.merge(output,customer_son_real_pay,on='customer_id')
    output=pd.merge(output,customer_son_happy_pay,on='customer_id')
    print("现在人的特征我把你merge上了")
    
    # 统计这个用户的订单最后一次购买时间
    last_time = date1.groupby(['customer_id'],as_index=False)['order_pay_date'].agg({'order_pay_date_last':'max',
                                                                                     'order_pay_date_first':'min'}
                                                                                    )
    
    
    
    #购买时间间隔 这个地方不要提取错了
    date1.sort_values(by='order_pay_time',inplace=True)
    q=date1.groupby(['customer_id'], as_index=False)['order_pay_time'].agg({'deta_time_mean':(lambda x:x.diff().mean()),
                                                                           'deta_time_max':(lambda x:x.diff().max()),
                                                                           'deta_time_min':(lambda x:x.diff().min())
                                                                          })
    #要是中间就买了一次 我们认为时间间隔为0
    q.fillna(0,inplace=True)
    q['deta_time_mean']=q['deta_time_mean'].dt.days
    q['deta_time_max']=q['deta_time_max'].dt.days
    q['deta_time_min']=q['deta_time_min'].dt.days
    
    output=pd.merge(output,last_time,on='customer_id')
    output=pd.merge(output,q,on='customer_id')
    print('时间也已经把你拼接上去了')
    
    
    #return output
    print('下面开始地域特征拼接')
    Location=date1.groupby(['customer_id','new_location_code'])['new_location_code'].agg({'最多来自哪里':np.size})
    Location=Location.reset_index().sort_values(by='最多来自哪里',ascending=False)
    Location=Location.drop_duplicates('customer_id')
    Location=Location[['customer_id','最多来自哪里']]
    
    print('下面获取经纬度数据')
    
    Longtitude = date1.groupby(['customer_id'], as_index=False)['精度'].agg(
                                                                              {  '精度_max':'max',
                                                                                 '精度_mean':'mean',
                                                                                 '精度_min':'min',
                                                                                 '精度_median':'median'}
                                                                                              )

    Latitude = date1.groupby(['customer_id'], as_index=False)['纬度'].agg(
                                                                              {  '纬度_max':'max',
                                                                                 '纬度_mean':'mean',
                                                                                 '纬度_min':'min',
                                                                                 '纬度_median':'median'}
                                                                                              )
    print('经纬度数据完毕')
    
   
    output=pd.merge(output,Location,on='customer_id')
    output=pd.merge(output,Longtitude,on='customer_id')
    output=pd.merge(output,Latitude,on='customer_id')
    
    print("至此 全部的数据拼接完毕")

    print("进入提交环节")

    if isSubmit==False:
        print("您选择的是本地训练模式")
        output['order_pay_date_jiange']=(output['order_pay_date_last']-output['order_pay_date_first']).dt.days
        output['label'] = 0
        output.loc[output['customer_id'].isin(list(date2['customer_id'].unique())),'label'] = 1
        output=output.drop(columns=['order_pay_date_last','order_pay_date_first'])
    else:
        print('您选择的是合并所有集合模式')
        output['order_pay_date_jiange'] = pd.to_datetime('2013-12-31') - pd.to_datetime(output['order_pay_date_last'])
        output['order_pay_date_jiange'] = output['order_pay_date_jiange'].dt.days + 1
        output=output.drop(columns=['order_pay_date_last','order_pay_date_first'])
    
    print("合并完成，现在开始输出")
    print(output.shape)
    return output



In [ ]:

#划分训练集和全部集合
# 生成训练数据和提交数据
train_history = all_data[(all_data['order_pay_date'].astype(str)<='2013-11-12')]
online_history = all_data[(all_data['order_pay_date'].astype(str)<='2013-12-31')]
train_label = all_data[all_data['order_pay_date'].astype(str)>='2013-07-04']


train = make_feature_and_label(train_history,train_label,False)
#submit = make_feature_and_label(online_history,None,True)


In [ ]:
train.to_csv('my_train_change_big.csv')
print("文件生成完毕")

In [ ]:
submit = make_feature_and_label(online_history,None,True)

In [ ]:
train.shape

In [ ]:
submit.shape

In [ ]:
#submit 是my_all_change
submit.to_csv('my_all_change.csv')
train.to_csv('my_train_change.csv')

In [ ]:
#output=pd.read_csv('my_train.csv')
#print(output.shape)
#print(output.columns)

In [ ]:
import lightgbm as lgb



def logloss(y_true, y_pred,deta = 3.4, eps=1e-15):
    # Prepare numpy array data
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    assert (len(y_true) and len(y_true) == len(y_pred))
    # Clip y_pred between eps and 1-eps
    p = np.clip(y_pred, eps, 1-eps)
    loss = np.sum(- y_true * np.log(p) * deta - (1 - y_true) * np.log(1-p))
    return loss / len(y_true)



param = {
    'num_leaves':128,
    'objective':'binary',
    'max_depth':-1,
    'learning_rate':0.1,
    'metric':'binary_logloss',
    'bagging_fraction':0.95,
    'bagging_freq':5,
    "device_type ":'gpu',
    'gpu_use_dp':True,
    "scale_pos_weight":5

}
# 构建机器学习所需的label和data



feature = [x for x in train.columns if x not in ['customer_id']]
print("最终提取的特征是：",feature)

In [ ]:
train.head()

In [ ]:
y.value_counts()

In [ ]:
from sklearn.model_selection import train_test_split

X = train[feature]
# 划分训练集和验证集
print(X.shape)

In [ ]:
import numpy as np

def log_loss(y_pred, data_vail):
    
    deta = 3.4
    eps=1e-15
    
    labels = data_vail.get_label()
    p = np.clip(y_pred, eps, 1-eps)
    loss = np.sum(- labels * np.log(p) * deta - (1 - labels) * np.log(1-p))
    return 'log_loss', loss/ len(labels), False


param = {
    'num_leaves':128,
    'objective':'binary',
    'max_depth':-1,
    'learning_rate':0.1,
    #'metric':'binary_logloss',
    'bagging_fraction':0.95,
    'bagging_freq':5,
    "device_type ":'gpu',
    'gpu_use_dp':True,
    "scale_pos_weight":5

}

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=42,stratify=y)


#
trn_data = lgb.Dataset(X_train, label=y_train)
val_data = lgb.Dataset(X_valid, label=y_valid)

lgbm = lgb.train(param,
                 trn_data,
                 valid_sets=[trn_data,val_data],
                 num_boost_round = 100 ,
                 #early_stopping_rounds=1000,
                 learning_rates=lambda iter: 0.05 * (0.99 ** iter),
                 feval=log_loss
                 )

In [ ]:

def logloss(y_true, y_pred,deta = 3.4, eps=1e-15):
    # Prepare numpy array data
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    assert (len(y_true) and len(y_true) == len(y_pred))
    # Clip y_pred between eps and 1-eps
    p = np.clip(y_pred, eps, 1-eps)
    loss = np.sum(- y_true * np.log(p) * deta - (1 - y_true) * np.log(1-p))
    return loss / len(y_true)


y_pred1 = lgbm.predict(X_valid,num_iteration=lgbm.best_iteration)
error1 = logloss(y_valid,y_pred1)
print('验证集上的误差为：',error1)

y_pred2 = lgbm.predict(X_train,num_iteration=lgbm.best_iteration)
error2 = logloss(y_train,y_pred2)
#mse2.append(error2)
print('训练集上的误差为：',error2)

In [ ]:

submit_df = submit[['customer_id']]
X_submit = submit[feature]





In [ ]:
X_submit.shape

In [ ]:
q_submit=X_submit[0:200000]
y1_submit = lgbm.predict(q_submit)
del q_submit
q_submit=X_submit[200000:300000]
y2_submit = lgbm.predict(q_submit)
del q_submit
q_submit=X_submit[400000:600000]
y2_submit = lgbm.predict(q_submit)
del q_submit
q_submit=X_submit[600000:929806]
y3_submit = lgbm.predict(q_submit)
del q_submit

In [ ]:
q_submit=X_submit[200000:300000]
y2_submit = lgbm.predict(q_submit)
del q_submit

In [ ]:
y_submit

In [ ]:
y_submit = lgbm.predict(q_submit)



In [ ]:
q_submit.shape

In [ ]:
X_submit.shape

In [ ]:
all_customer = pd.merge(all_customer,submit_df,on=['customer_id'],how='left',copy=False)
all_customer = all_customer.sort_values(['customer_id'])
all_customer['customer_id'] = all_customer['customer_id'].astype('int64')
all_customer['result'] = all_customer['result'].fillna(0)
all_customer.to_csv('./mpdf_baseline.csv',index=False)

In [ ]:

submit_df = submit[['customer_id']]
X_submit = submit[feature]
y_submit = lgbm.predict(X_submit)

submit_df['result'] = y_submit